<a href="https://colab.research.google.com/github/Kr1n3/MPC_2022/blob/main/Fashion_Items_Classification_Yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ultralytics/yolov5  
%cd yolov5
%pip install -qr requirements.txt  

import torch
import utils
display = utils.notebook_init()  

YOLOv5 🚀 v6.2-224-g82a5585 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.9/78.2 GB disk)


In [2]:
import os
os.makedirs("../datasets/", exist_ok=True)
%cd ../datasets/

/content/datasets


In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="rMzR9eoIcBQzHYZV8kcV")
project = rf.workspace("pos").project("clothing-classification-4xbvj")
dataset = project.version(2).download("folder")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 219 kB/s 
     |████████████████████████████████| 178 kB 64.8 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 145 kB 71.4 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 138 kB 71.5 MB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=69e94c69089db51fe897d7c1a4956006962440d6c42ba0385673d144c75e6f32
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: certifi
    Found existing installation: certi

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Clothing-Classification-2 in folder:: 100%|██████████| 659/659 [00:00<00:00, 1442.37it/s]


In [4]:
os.rename("/content/datasets/Clothing-Classification-2/valid", "/content/datasets/Clothing-Classification-2/val")

In [5]:
%cd ../yolov5
from utils.downloads import attempt_download

p5 = ['Bag', 'Dress', 'Pants', 'Shoes', 'Skirt']  
cls = [f'{x}-cls' for x in p5] 

for x in cls:
    attempt_download(f'weights/yolov5{x}.pt')

/content/yolov5


In [6]:
dataset.location

'/content/datasets/Clothing-Classification-2'

In [7]:
Fashion = dataset.location.split(os.sep)[-1]
os.environ["DATASET_NAME"] = Fashion

In [8]:
# Weights & Biases  
%pip install -q wandb
import wandb

     |████████████████████████████████| 1.9 MB 31.7 MB/s 
     |████████████████████████████████| 182 kB 72.5 MB/s 
     |████████████████████████████████| 166 kB 71.4 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 140 kB 69.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
roboflow 0.2.18 requires urllib3==1.26.6, but you have urllib3 1.26.12 which is incompatible.


In [10]:
import sys

def wandb_colab_login():
  """Temporary hack to prevent colab from hanging"""
  sys.modules["google.colab2"] = sys.modules["google.colab"]
  del sys.modules["google.colab"]
  wandb.login()
  sys.modules["google.colab"] = sys.modules["google.colab2"]
wandb_colab_login()

wandb: Currently logged in as: kr1n3. Use `wandb login --relogin` to force relogin


In [11]:
!python classify/train.py --model yolov5s-cls.pt --data $Fashion --epochs 300 --img 640  --pretrained weights/yolov5s-cls.pt


wandb: Currently logged in as: kr1n3. Use `wandb login --relogin` to force relogin
classify/train: model=yolov5s-cls.pt, data=Clothing-Classification-2, epochs=300, batch_size=64, imgsz=640, nosave=False, cache=None, device=, workers=8, project=runs/train-cls, name=exp, exist_ok=False, pretrained=weights/yolov5s-cls.pt, optimizer=Adam, lr0=0.001, decay=5e-05, label_smoothing=0.1, cutoff=None, dropout=None, verbose=False, seed=0, local_rank=-1
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-224-g82a5585 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

TensorBoard: Start with 'tensorboard --logdir runs/train-cls', view at http://localhost:6006/
wandb: Tracking run with wandb version 0.13.4
wandb: Run data is saved locally in /content/yolov5/wandb/run-20221102_190545-3mjmx41j
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run super-hill-12
wandb: ⭐️ View project at https://wandb.ai/kr1n3/YOLOv5-Classify
wandb: 🚀 View run at https:

In [12]:
!python classify/val.py --weights runs/train-cls/exp/weights/best.pt --data /content/datasets/Clothing-Classification-2


classify/val: data=/content/datasets/Clothing-Classification-2, weights=['runs/train-cls/exp/weights/best.pt'], batch_size=128, imgsz=224, device=, workers=8, verbose=True, project=runs/val-cls, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.2-224-g82a5585 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 117 layers, 4173093 parameters, 0 gradients, 10.4 GFLOPs
validating: 100% 1/1 [00:00<00:00,  1.62it/s]
                   Class      Images    top1_acc    top5_acc
                     all          51       0.294           1
                     Bag           5         0.6           1
                   Dress          13       0.538           1
                   Pants          12       0.167           1
                   Shoes          12       0.167           1
                   Skirt           9       0.111           1
Speed: 0.1ms pre-process, 2.1ms inference, 0.0ms post-process per image at shape (1, 3, 224, 224)
Results

In [13]:
#export your model's weights for future use
from google.colab import files
files.download('./runs/train-cls/exp/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>